# Reasoning practice

Set up your API in .env file 

In [6]:
import os
from dotenv import load_dotenv
from IPython.display import Markdown
import google.generativeai as genai
from pathlib import Path

load_dotenv()

# Colab 환경 체크
try:
    from google.colab import userdata
    IN_COLAB = True
except ImportError:
    IN_COLAB = False

# API 키 설정
if IN_COLAB:
    # Colab 환경에서 API 키 불러오기
    API_KEY = userdata.get("GOOGLE_API_KEY")
else:
    # 로컬 환경에서는 dotenv 사용
    try:
        from dotenv import load_dotenv
        # 현재 노트북이 있는 디렉토리에서 .env 파일 찾기
        load_dotenv()  # 현재 디렉토리 또는 상위 디렉토리에서 .env 자동 탐색
    except ImportError:
        print("python-dotenv가 설치되지 않았습니다. 환경변수에서 직접 읽습니다.")
    
    # 환경변수에서 키 읽기
    API_KEY = os.getenv('GOOGLE_API_KEY')
    if not API_KEY:
        raise ValueError("GOOGLE_API_KEY 환경변수를 설정해주세요")

# API 키 설정
genai.configure(api_key=API_KEY)
MODEL = "gemini-2.0-flash-exp"  # 또는 "gemini-1.5-flash", "gemini-1.5-pro"

# 모델 초기화
model = genai.GenerativeModel(MODEL)

# Gemini 호출 함수 (안전한 텍스트 추출 포함)
def call_gemini(prompt: str, thinking: int = 0, max_tokens: int = 600, temperature: float = 0.2) -> str:
    try:
        generation_config = genai.GenerationConfig(
            max_output_tokens=max_tokens,
            temperature=temperature,
        )
        
        response = model.generate_content(
            prompt,
            generation_config=generation_config
        )
        
        # 응답 텍스트 추출
        if response.text:
            return response.text.strip()
        else:
            return "(No text returned.)"
            
    except Exception as e:
        return f"Error: {str(e)}"


# --- 프롬프트 (예시: 학생 풀이 채점) ---
prompt = """
Determine if the student's solution is correct or not. 

Question:
I'm building a solar power installation and I need help working out the financials.
- Land costs 250 / square foot
- I negotiated a contract for maintenance that will cost me a flat 10 / square foot
What is the total cost for the first year of operations as a function of the number of square feet.

Student's Solution:
Let x be the size of the installation in square feet.
Costs:
1. Land cost: 100x
2. Solar panel cost: 250x
3. Maintenance cost: 100,000 + 100x
Total cost: 100x + 250x + 100,000 + 100x = 450x + 100,000

Explain your reasoning step by step in Korean.
"""

#Explain your reasoning step by step in Korean 도 프롬프트 끝단에 추가해보기 

# --- 실행 ---
response = call_gemini(prompt)
display(Markdown(response))

E0000 00:00:1758445105.741364 59803079 alts_credentials.cc:93] ALTS creds ignored. Not running on GCP and untrusted ALTS is not enabled.


학생의 풀이가 맞는지 단계별로 설명하겠습니다.

**1. 문제 이해:**

문제가 요구하는 것은 태양광 발전 시설의 첫 해 운영 비용을 시설 면적(제곱피트)의 함수로 나타내는 것입니다.  주어진 정보는 다음과 같습니다:

*   땅값: 제곱피트당 250
*   유지 보수 비용: 제곱피트당 10

**2. 학생 풀이 분석:**

학생의 풀이에는 몇 가지 오류가 있습니다.

*   **땅값:** 학생은 땅값을 100x라고 계산했습니다. 문제에서 땅값은 제곱피트당 250이라고 주어졌으므로, 땅값은 250x가 되어야 합니다.
*   **태양광 패널 비용:** 문제에는 태양광 패널 비용에 대한 정보가 없습니다. 따라서 이 비용은 계산에 포함되어서는 안 됩니다.
*   **유지 보수 비용:** 학생은 유지 보수 비용을 100,000 + 100x라고 계산했습니다. 문제에서 유지 보수 비용은 제곱피트당 10이라고 주어졌으므로, 유지 보수 비용은 10x가 되어야 합니다. 100,000이라는 고정 비용은 문제에 제시되지 않았습니다.

**3. 올바른 풀이:**

*   x를 시설 면적 (제곱피트)라고 합시다.
*   땅값: 250x
*   유지 보수 비용: 10x
*   총 비용: 250x + 10x = 260x

따라서, 첫 해 운영 비용은 260x입니다.

**4. 결론:**

학생의 풀이는 땅값, 태양광 패널 비용, 유지 보수 비용 계산에 오류가 있어 틀렸습니다. 올바른 답은 260x입니다.

**한국어 요약:**

학생의 풀이는 틀렸습니다. 문제에서 주어진 정보를 잘못 해석하고, 불필요한 비용을 추가했습니다. 땅값은 제곱피트당 250이므로 250x로 계산해야 하고, 유지 보수 비용은 제곱피트당 10이므로 10x로 계산해야 합니다. 태양광 패널 비용은 문제에 언급되지 않았으므로 포함해서는 안 됩니다. 따라서, 올바른 답은 260x입니다.

Other tasks

### [문제 1] 조건부 확률 (베이즈) 채점 예시

In [7]:
# ================== [문제 1] 조건부 확률 (베이즈) 채점 예시 ==================
# 이 셀은 독립 실행 가능하도록 구성했습니다. (설정 + 함수 + 프롬프트 + 실행)
# 과제 지시: 학생의 풀이가 맞는지 평가하세요. (조건부 확률/검사 양성 시 질병 확률)

from IPython.display import Markdown
import os
import google.generativeai as genai
from pathlib import Path

# Colab 환경 체크
try:
    from google.colab import userdata
    IN_COLAB = True
except ImportError:
    IN_COLAB = False

# API 키 설정
if IN_COLAB:
    API_KEY = userdata.get("GOOGLE_API_KEY")
else:
    try:
        from dotenv import load_dotenv
        # 현재 디렉토리에서 .env 파일 자동 탐색
        load_dotenv()
    except ImportError:
        print("python-dotenv가 설치되지 않았습니다. 환경변수에서 직접 읽습니다.")
    API_KEY = os.getenv('GOOGLE_API_KEY')
    if not API_KEY:
        raise ValueError("GOOGLE_API_KEY 환경변수를 설정해주세요")

genai.configure(api_key=API_KEY)
MODEL = "gemini-2.0-flash-exp"
model = genai.GenerativeModel(MODEL)

# --- 안전한 호출 함수 (resp.text가 None일 때 대비) ---
def call_gemini(prompt: str, thinking: int = 0, max_tokens: int = 600, temperature: float = 0.2) -> str:
    try:
        generation_config = genai.GenerationConfig(
            max_output_tokens=max_tokens,
            temperature=temperature,
        )
        
        response = model.generate_content(
            prompt,
            generation_config=generation_config
        )
        
        if response.text:
            return response.text.strip()
        else:
            return "(No text returned.)"
    except Exception as e:
        return f"Error: {str(e)}"

# --- 프롬프트 (조건부 확률 과제) ---
# 상황: 질병 유병률 1%, 검사 민감도 90%, 특이도 90%.
# 질문: 검사 양성일 때 실제로 질병일 확률 P(Disease|Positive)?
# 오답 예시: 단순히 90%라고 주장하는 베이즈 오류(기저율 무시).

#Determine if the student's solution is correct or not.

prompt = """
Question:
A disease has a 1% prevalence in the population. A diagnostic test has 90% sensitivity
(P(Positive|Disease)=0.9) and 90% specificity (P(Negative|NoDisease)=0.9).
If a person tests positive, what is P(Disease|Positive)?

Answer in Korean

Student's Solution:
Because the test is 90% accurate, P(Disease|Positive) is 90%.

Task:
- State whether the student's answer is correct or incorrect.
- Provide a brief correct calculation using Bayes' rule in plain English.
- Give a one-sentence explanation of the common cognitive mistake involved (e.g., base-rate neglect).
"""

# --- 실행 ---
response = call_gemini(prompt, thinking=1) #thinking = 1로 해보고 비교하기
display(Markdown(response))

E0000 00:00:1758445284.651000 59803079 alts_credentials.cc:93] ALTS creds ignored. Not running on GCP and untrusted ALTS is not enabled.


## 분석 및 해설

**학생의 답변:** 틀렸습니다.

**올바른 계산 (베이즈 정리 사용):**

1. **사전 확률:** 질병 유병률은 1%이므로 P(Disease) = 0.01, P(NoDisease) = 0.99 입니다.
2. **민감도 및 특이도:** 민감도는 90%이므로 P(Positive|Disease) = 0.9, 특이도는 90%이므로 P(Negative|NoDisease) = 0.9 입니다. 따라서 P(Positive|NoDisease) = 1 - 0.9 = 0.1 입니다.
3. **베이즈 정리 적용:** P(Disease|Positive) = [P(Positive|Disease) * P(Disease)] / [P(Positive|Disease) * P(Disease) + P(Positive|NoDisease) * P(NoDisease)] = (0.9 * 0.01) / (0.9 * 0.01 + 0.1 * 0.99) = 0.009 / (0.009 + 0.099) = 0.009 / 0.108 = 0.0833 (약 8.33%) 입니다.

따라서, 양성 판정을 받은 사람이 실제로 질병을 가지고 있을 확률은 약 8.33% 입니다.

**흔한 인지적 오류 설명:**

학생은 기저율 무시 오류를 범했는데, 이는 질병의 낮은 유병률을 고려하지 않고 검사의 정확도만을 맹신하여 실제 질병을 가지고 있을 확률을 과대평가하는 오류입니다.

[문제 2] 휴리스틱/연합오류(결합오류; Conjunction Fallacy) 채점 예시

1. 단순히 프롬프트 결과를 보려면 채점 예시 없이 실행하고 결과 확인

2. 프롬프트 체이닝을 보려면 학생의 답을 임의로 입력하고 학생 채점 결과도 같이 확인

In [8]:
from IPython.display import Markdown
import os
import google.generativeai as genai
from pathlib import Path

# Colab 환경 체크
try:
    from google.colab import userdata
    IN_COLAB = True
except ImportError:
    IN_COLAB = False

# API 키 설정
if IN_COLAB:
    API_KEY = userdata.get("GOOGLE_API_KEY")
else:
    try:
        from dotenv import load_dotenv
        load_dotenv()  # 현재 디렉토리에서 .env 파일 자동 탐색
    except ImportError:
        print("python-dotenv가 설치되지 않았습니다. 환경변수에서 직접 읽습니다.")
    API_KEY = os.getenv('GOOGLE_API_KEY')
    if not API_KEY:
        raise ValueError("GOOGLE_API_KEY 환경변수를 설정해주세요")

genai.configure(api_key=API_KEY)
MODEL = "gemini-2.0-flash-exp"
model = genai.GenerativeModel(MODEL)

# --- 안전한 호출 함수 ---
def call_gemini(prompt: str, thinking: int = 0, max_tokens: int = 600, temperature: float = 0.2) -> str:
    try:
        generation_config = genai.GenerationConfig(
            max_output_tokens=max_tokens,
            temperature=temperature,
        )
        
        response = model.generate_content(
            prompt,
            generation_config=generation_config
        )
        
        if response.text:
            return response.text.strip()
        else:
            return "(No text returned.)"
    except Exception as e:
        return f"Error: {str(e)}"

# --- 프롬프트 (결합오류 과제) ---
# 상황: 대표성 휴리스틱으로 유명한 Linda 문제 변형. '둘 다' 사건이 단독 사건보다 확률이 더 크다고 판단하는 오류.
prompt = """
Determine if the student's solution is correct or not.

Question (Conjunction Fallacy):
Linda is described as deeply concerned with social justice and active in community issues.
Which is more probable?
A) Linda is a bank teller.
B) Linda is a bank teller and a feminist.

Student's Answer:
B) is more probable.

Task:
- Judge the student's answer as correct or incorrect.
- Provide a short explanation referencing the conjunction rule (P(A and B) ≤ P(A)).
- Briefly name the heuristic involved (e.g., representativeness) in one sentence.
"""

# --- 실행 ---
response = call_gemini(prompt, thinking=0)
display(Markdown(response))

E0000 00:00:1758445306.395207 59803079 alts_credentials.cc:93] ALTS creds ignored. Not running on GCP and untrusted ALTS is not enabled.


**Judgment:** Incorrect

**Explanation:** The conjunction rule states that the probability of two events occurring together (A and B) cannot be higher than the probability of either event occurring alone (P(A and B) ≤ P(A)). Therefore, it is always more probable that Linda is simply a bank teller than that she is a bank teller *and* a feminist.

**Heuristic:** The representativeness heuristic leads people to judge the probability of an event based on how similar it is to a stereotype or mental image.

In [16]:
# ================== [문제 3] DIY 실습용 템플릿 (직접 문제/풀이 작성) ==================
# 이 셀은 학생이 직접  주제를 정하고,
# 문제/학생 답안을 채워 넣어 채점 프롬프트를 완성하도록 구성했습니다.
# 아래 TODO들을 자신의 주제로 바꾼 뒤 실행하세요.

from IPython.display import Markdown
import os
import google.generativeai as genai
from pathlib import Path

# Colab 환경 체크
try:
    from google.colab import userdata
    IN_COLAB = True
except ImportError:
    IN_COLAB = False

# API 키 설정
if IN_COLAB:
    API_KEY = userdata.get("GOOGLE_API_KEY")
else:
    try:
        from dotenv import load_dotenv
        load_dotenv()  # 현재 디렉토리에서 .env 파일 자동 탐색
    except ImportError:
        print("python-dotenv가 설치되지 않았습니다. 환경변수에서 직접 읽습니다.")
    API_KEY = os.getenv('GOOGLE_API_KEY')
    if not API_KEY:
        raise ValueError("GOOGLE_API_KEY 환경변수를 설정해주세요")

genai.configure(api_key=API_KEY)
MODEL = "gemini-2.0-flash-exp"
model = genai.GenerativeModel(MODEL)

# --- 안전한 호출 함수 ---
def call_gemini(prompt: str, thinking: int = 0, max_tokens: int = 600, temperature: float = 0.2) -> str:
    try:
        generation_config = genai.GenerationConfig(
            max_output_tokens=max_tokens,
            temperature=temperature,
        )
        
        response = model.generate_content(
            prompt,
            generation_config=generation_config
        )
        
        if response.text:
            return response.text.strip()
        else:
            return "(No text returned.)"
    except Exception as e:
        return f"Error: {str(e)}"

# === TODO: 아래 세 블록을 자신의 주제로 교체하세요 ===
DIY_QUESTION = """
A smoker in a quitting program expects a reward of 5 points for resisting a cigarette (expected value V(s)=5).
However, in this trial the participant receives only 2 points as actual reward (r=2).

Question:
What is the reward prediction error (RPE) in this case?
"""

DIY_STUDENT_SOLUTION = """
I think the RPE is +7, because r + V(s) = 2 + 5 = 7.
"""

DIY_TASK_INSTRUCTIONS = """
Task:
- State whether the student's answer is correct or incorrect.
- Provide the correct calculation of RPE in plain language (use Korean, only use English for key terms).
- Briefly explain why the error occurred.
- Keep feedback within 5–8 lines.
- Suggest possible teaching strategies for the student to improve their understanding. If needed, generate a possible feedback message or additional quiz questions for the student.
"""

# --- 프롬프트 생성 ---
prompt = f"""
Determine if the student's solution is correct or not.

Question:
{DIY_QUESTION}

Student's Solution:
{DIY_STUDENT_SOLUTION}

{DIY_TASK_INSTRUCTIONS}
"""

# --- 실행 (필요 시 reasoning을 켜고 싶다면 thinking=1~4 정도로 조절) ---
response = call_gemini(prompt, thinking=1)
display(Markdown(response))

E0000 00:00:1758445943.037354 59803079 alts_credentials.cc:93] ALTS creds ignored. Not running on GCP and untrusted ALTS is not enabled.


학생의 답은 **틀렸습니다**.

Reward Prediction Error (RPE)는 실제 보상(r)에서 예상 보상(V(s))을 뺀 값입니다. 즉, RPE = r - V(s) = 2 - 5 = -3 입니다.

학생은 실제 보상과 예상 보상을 더하는 실수를 했습니다. RPE는 예상과 실제의 차이를 나타내므로 빼야 합니다.

학생이 RPE의 정의를 혼동한 것 같습니다. RPE 계산 공식을 명확히 이해하도록 반복 학습이 필요합니다. 예를 들어, "만약 예상 점수가 10점이고 실제 받은 점수가 7점이라면 RPE는 얼마일까요?"와 같은 추가 질문을 통해 이해도를 높일 수 있습니다.

[optional] Example of prompt chaining

In [ ]:
# ============================================
# (1) Instructor Demo: Prompt Chaining
# ============================================

# Step 1: 기억의 종류 설명
explanation = call_gemini(
    "Explain major types of memory (sensory, short-term/working, long-term: episodic, semantic, procedural) "
    "for intro psychology students in about 120 words with daily-life examples."
)
print("=== Step 1: Explanation ===")
print(explanation, "\n")

# Step 2: 설명을 기반으로 퀴즈 질문 만들기
quiz = call_gemini(
    f"Create 3 quiz questions for students based on this explanation. "
    f"Mix MCQ and short-answer, and tag difficulty (Easy/Med/Hard).\n\nExplanation:\n{explanation}"
)
print("=== Step 2: Quiz Questions ===")
print(quiz, "\n")

# Step 3: 퀴즈 정답 만들기
answers = call_gemini(
    f"Provide a short answer key (1–2 lines each) for the following quiz questions:\n\n{quiz}"
)
print("=== Step 3: Answer Key ===")
print(answers)


=== Step 1: Explanation ===
Imagine your mind as a bustling office.

**Sensory Memory** is like a fleeting glance at a document – you briefly see everything (e.g., a flash of a car's color as it speeds by), but it's gone in seconds unless you pay attention.

**Short-Term/Working Memory** is your desk's "inbox" – you can actively hold a few items and manipulate them (like remembering a phone number just long enough to dial it, or mentally calculating a tip). It's temporary and limited.

**Long-Term Memory** is the vast filing cabinet.
*   **Episodic Memory** stores personal "episodes" – like remembering your last birthday party or what you ate for breakfast.
*   **Semantic Memory** holds general facts and knowledge – knowing that Paris is the capital of France or what a "dog" is.
*   **Procedural Memory** is "muscle memory" – knowing *how* to ride a bike, tie your shoes, or type without thinking. These are skills you perform automatically. 

=== Step 2: Quiz Questions ===
Here are 3 qui

DIY non-reasoning

In [ ]:
# ============================================
# (2) Student Lab: Non-chaining (One-shot)
# ============================================

oneshot_prompt = (
    "In one response, do ALL of the following on 'types of memory':\n"
    "1) Explain sensory, short-term/working, and long-term (episodic, semantic, procedural) with daily examples (≤150 words).\n"
    "2) Make 3 quiz questions (MCQ + short-answer, tag difficulty).\n"
    "3) Provide a short answer key."
)

oneshot_output = call_gemini(oneshot_prompt, thinking=0)
print("=== One-shot Output (non-reasoning) ===")
print(oneshot_output)


=== One-shot Output (non-reasoning) ===
Here's your comprehensive response on types of memory:

1) **Memory Types Explained:**
Our memory system is a fascinating process. **Sensory memory** is the initial, fleeting capture of sensory information – like the afterimage of a bright flash or the echo of a sound. It lasts only milliseconds to a few seconds. **Short-term memory (STM)**, often used interchangeably with **working memory**, holds a small amount of information (around 7 items) for about 15-30 seconds, allowing us to actively process it, like remembering a phone number just long enough to dial. **Long-term memory (LTM)** has a vast, potentially unlimited capacity and duration. Within LTM:
    *   **Episodic memory** stores personal experiences and events (e.g., your last birthday party).
    *   **Semantic memory** holds general knowledge and facts (e.g., the capital of France).
    *   **Procedural memory** stores how to do things, skills and habits (e.g., riding a bike, typing)

In [ ]:
# ============================================
# (3) Student Lab: Chaining + Reasoning
# ============================================

# Step A: 설명 (reasoning 활성)
stepA = call_gemini(
    "Explain the main types of memory (sensory, short-term/working, long-term: episodic, semantic, procedural) "
    "with daily examples in ~120 words.", thinking=8
)
print("=== Step A: Explanation ===")
print(stepA, "\n")

# Step B: 퀴즈 (Step A 결과 활용)
stepB = call_gemini(
    f"Based on the explanation below, create 3 quiz questions. "
    f"Mix MCQ and short-answer, tag difficulty.\n\n{stepA}", thinking=8
)
print("=== Step B: Quiz Questions ===")
print(stepB, "\n")

# Step C: 정답 (Step B 결과 활용)
stepC = call_gemini(
    f"Provide a concise answer key (1–2 lines each) for these questions:\n\n{stepB}", thinking=8
)
print("=== Step C: Answer Key ===")
print(stepC)


=== Step A: Explanation ===
Memory types include:

**Sensory memory** briefly holds incoming sensory information (e.g., seeing a flash of lightning, hearing a sudden noise). It lasts only a few seconds.

**Short-term memory (STM)** temporarily stores and processes limited information (around 7 items for ~30 seconds). **Working memory** is an active form of STM, allowing manipulation of information (e.g., remembering a phone number while dialing, mentally calculating a tip).

**Long-term memory (LTM)** stores vast amounts of information indefinitely.
*   **Episodic memory** stores personal experiences (e.g., your last birthday party).
*   **Semantic memory** stores general knowledge and facts (e.g., the capital of France).
*   **Procedural memory** stores how to do things (e.g., riding a bike, tying your shoes). 

=== Step B: Quiz Questions ===
Here are 3 quiz questions based on the provided explanation, mixing MCQ and short-answer, with difficulty tags:

---

**Question 1 (Multiple Cho

In [ ]:
# ============================================
# (4) Student Lab: Show Model Reasoning Only
# ============================================

# reasoning trace만 보고 싶을 때: thinking=1로 요청
# (주의: 모델/SDK 버전에 따라 reasoning 출력이 없을 수도 있음)

reasoning_demo = call_gemini(
    "Briefly explain differences among sensory, short-term, and long-term memory with one daily-life example.",
    thinking=1
)

print("=== Model Reasoning (if exposed) ===")
print(reasoning_demo if reasoning_demo else "(No reasoning output available)")


=== Model Reasoning (if exposed) ===
Here's a brief explanation:

*   **Sensory Memory:** The initial, fleeting capture of sensory information (sight, sound, touch) for a fraction of a second to a few seconds. It's largely unconscious.
*   **Short-Term Memory (STM):** A temporary storage system that holds a small amount of information (around 7 items) for about 15-30 seconds, unless actively rehearsed. It's where you consciously process information.
*   **Long-Term Memory (LTM):** A vast, relatively permanent storage system for information, skills, and experiences. It has an unlimited capacity and can last for a lifetime.

**Daily-Life Example:**

Imagine you're looking up a new pizza place's phone number:

1.  **Sensory Memory:** You glance at the phone number on the menu. For a split second, your eyes capture the entire visual image of the number, even before you consciously register it.
2.  **Short-Term Memory:** You then focus on the digits, repeating "555-1234" to yourself a few t

DIY

In [17]:
# - 각 TODO 부분에 본인이 관심 있는 주제나 프롬프트를 직접 작성한다.
# - 모든 코드는 순서대로 실행하며, 각 단계의 출력 차이를 비교한다.
#
# ============================================================

import os
import google.generativeai as genai
from pathlib import Path

# Colab 환경 체크
try:
    from google.colab import userdata
    IN_COLAB = True
except ImportError:
    IN_COLAB = False

# API 키 설정
if IN_COLAB:
    # Colab 환경에서 API 키 불러오기
    API_KEY = userdata.get("GOOGLE_API_KEY")
else:
    # 로컬 환경에서는 dotenv 사용
    try:
        from dotenv import load_dotenv
        load_dotenv()  # 현재 디렉토리에서 .env 파일 자동 탐색
    except ImportError:
        print("python-dotenv가 설치되지 않았습니다. 환경변수에서 직접 읽습니다.")
    API_KEY = os.getenv('GOOGLE_API_KEY')
    if not API_KEY:
        raise ValueError("GOOGLE_API_KEY 환경변수를 설정해주세요")

genai.configure(api_key=API_KEY)
MODEL = "gemini-2.0-flash-exp"  # 필요 시 "gemini-1.5-pro"로 변경 가능

model = genai.GenerativeModel(MODEL)

# Gemini 호출 함수 (간단한 wrapper)
def call_gemini(prompt: str, thinking: int = 0, max_tokens: int = 600, temperature: float = 0.2) -> str:
    """
    prompt: 모델에 전달할 문자열
    thinking: reasoning budget (0=비활성, 1 이상=간단 reasoning 활성)
    return: 모델의 출력 텍스트
    """
    generation_config = genai.GenerationConfig(
        max_output_tokens=max_tokens,
        temperature=temperature,
    )
    
    resp = model.generate_content(
        prompt,
        generation_config=generation_config
    )
    return resp.text

In [26]:
# ============================================================
# 실습 1 : 모델 reasoning 보기
# ============================================================

REASONING_PROMPT = """
- 주제: 강화학습(Reinforcement Learning)과 의사결정(Decision making)이 중독(addiction)과 어떤 관련이 있는지 설명하세요.
- **일상적 예시** 1개(흡연 금연 상황 등)를 포함하세요.
- **간단 수식** 1개를 본문에 한 줄로 넣으세요.
- 한국어로 작성하되, 핵심 용어는 영문 병기를 괄호로 함께 표기하세요.
- 출력 형식: 아래 템플릿을 따르세요.

## 출력 템플릿(Output Template)
#### RLDM과 addiction의 관계
- 정의(Definition): ...
- 핵심 수식(Key equation): δ = ...
- 관련 뇌 영역(Relevant brain areas): ...

#### 예시(Everyday Example)
- 상황: ...
- 해석: ...
"""

reasoning_output = call_gemini(REASONING_PROMPT, thinking=1)
print("=== Model Reasoning Output (if exposed) ===\n", reasoning_output or "(No reasoning trace exposed)")


=== Model Reasoning Output (if exposed) ===
 ## 강화학습과 의사결정이 중독에 미치는 영향

#### RLDM과 addiction의 관계
- 정의(Definition): 강화학습(Reinforcement Learning, RL)은 에이전트가 환경과의 상호작용을 통해 보상을 최대화하는 방향으로 학습하는 방법입니다. 의사결정(Decision Making, DM)은 이러한 학습 과정을 통해 최적의 행동을 선택하는 과정입니다. 중독(Addiction)은 특정 행동이나 물질에 대한 강박적인 추구와 사용으로, 뇌의 보상 시스템을 과도하게 활성화시켜 정상적인 의사결정 과정을 방해하는 상태입니다. 강화학습 모델은 중독 행동의 기저 메커니즘을 이해하고 설명하는 데 유용하게 사용될 수 있습니다. 중독은 강화학습 관점에서 볼 때, 특정 자극(예: 약물)에 대한 보상 예측 오류(Reward Prediction Error)가 과도하게 학습되어, 그 자극을 추구하는 행동이 강화되는 과정으로 설명될 수 있습니다. 즉, 기대했던 것보다 더 큰 보상을 경험하면, 그 행동을 반복할 가능성이 높아지고, 이러한 과정이 반복되면서 중독으로 이어질 수 있습니다.
- 핵심 수식(Key equation): 보상 예측 오류(Reward Prediction Error, RPE)는  δ = R + γV(s') - V(s) 로 표현됩니다. 여기서 R은 실제 받은 보상, γ는 할인율, V(s')는 다음 상태의 가치 함수, V(s)는 현재 상태의 가치 함수를 의미합니다.
- 관련 뇌 영역(Relevant brain areas): 복측피개영역(Ventral Tegmental Area, VTA), 측좌핵(Nucleus Accumbens, NAc), 전전두피질(Prefrontal Cortex, PFC) 등이 중독과 관련된 주요 뇌 영역입니다. VTA는 도파민을 분비하여 NAc에 전달하고, NAc는 보상과 관련된 감정을 처리합니다. PFC는 의사결정, 충동 조절, 실행 기능 등을 담당

In [30]:
# ============================================================
#  실습 2: Chaining + Reasoning (단계별)
# ============================================================
from IPython.display import Markdown, display
def show_md_block(text):
    display(Markdown(f"```\n{text}\n```"))

# Step A: DIY
STEP_A_PROMPT = """
- 주제: 강화학습(Reinforcement Learning)과 의사결정(Decision making)이 중독(addiction)과 어떤 관련이 있는지 설명하세요.
- 일상적 예시 1개(흡연 금연 상황 등)를 포함하세요.
- 간단 수식 1개를 본문에 한 줄로 넣으세요.
- 한국어로 작성하되, 핵심 용어는 영문 병기를 괄호로 함께 표기하세요.
- 출력 형식:

## RLDM과 addiction의 관계
- 정의(Definition): ...
- 핵심 수식(Key equation): δ = ...
- 관련 뇌 영역(Relevant brain areas): ...
#### 예시(Everyday Example)
- 상황: ...
- 해석: ...
"""
step_a = call_gemini(STEP_A_PROMPT, thinking=8)  # reasoning 활성화
print("=== Step A: Explanation ===")
show_md_block(step_a)
# print("=== Step A: Explanation ===\n", step_a, "\n")

# Step B: DIY
STEP_B_PROMPT = f"""
위 설명({step_a})을 바탕으로,
- RLDM 관련 주요 computational mechanism 2–3개를 뽑아라.
- 각 메커니즘에 대해 대응되는 중독 관련 임상적 특징(clinical feature)을 1줄로 설명하라.
- 출력 형식:

## Computational Mechanisms × Clinical Features
- Mechanism 1: ...
  - Clinical Feature: ...
- Mechanism 2: ...
  - Clinical Feature: ...
- Mechanism 3: ...
  - Clinical Feature: ...
"""
step_b = call_gemini(STEP_B_PROMPT, thinking=8)
print("=== Step B: Explanation ===")
show_md_block(step_b)
# print("=== Step B: Quiz Questions ===\n", step_b, "\n")

# Step C: DIY
STEP_C_PROMPT = f"""
위 메커니즘–임상 특징 매핑({step_b})을 기반으로,
- 향후 연구 질문(Research Questions) 2개를 제시하고 각 주제의 중요성을 어필하라. 각 항목은 5줄 이내 설명으로 작성하라.

## Research Directions
- Q1: ...
- Q2: ...
"""
step_c = call_gemini(STEP_C_PROMPT, thinking=8)
print("=== Step C: Explanation ===")
show_md_block(step_c)
# print("=== Step C: Answer Key ===\n", step_c, "\n")



=== Step A: Explanation ===


```
## RLDM과 addiction의 관계

- **정의 (Definition):** 강화학습 (Reinforcement Learning, RL)은 에이전트가 환경과의 상호작용을 통해 보상을 최대화하는 방향으로 학습하는 방법입니다. 의사결정 (Decision making)은 다양한 선택지 중에서 최적의 행동을 선택하는 과정이며, 강화학습은 이러한 의사결정 과정을 모델링하고 최적화하는 데 사용될 수 있습니다. 중독 (Addiction)은 특정 행동이나 물질에 대한 강박적인 추구와 사용을 특징으로 하는 뇌 질환입니다. 강화학습 관점에서 중독은 특정 행동이 주는 즉각적인 보상에 과도하게 집중하고, 장기적인 부정적 결과는 간과하는 학습 과정으로 해석될 수 있습니다. 즉, 중독은 비정상적인 강화학습 및 의사결정 과정의 결과로 볼 수 있습니다.

- **핵심 수식 (Key equation):** 시간차 오차 (Temporal Difference error)는 강화학습에서 중요한 개념으로, 예측된 보상과 실제 보상 간의 차이를 나타내며, 학습의 방향을 결정합니다. 시간차 오차는 다음과 같이 표현할 수 있습니다: δ = R + γV(s') - V(s), 여기서 R은 보상, γ는 할인율, V(s)는 현재 상태 s의 가치, V(s')는 다음 상태 s'의 가치를 의미합니다.

- **관련 뇌 영역 (Relevant brain areas):** 중독과 관련된 뇌 영역은 주로 보상 시스템 (Reward system)과 관련이 있습니다. 여기에는 복측피개영역 (Ventral Tegmental Area, VTA), 측좌핵 (Nucleus Accumbens, NAc), 전전두피질 (Prefrontal Cortex, PFC) 등이 포함됩니다. VTA는 도파민을 분비하여 NAc에 전달하고, NAc는 쾌락과 보상에 대한 감정을 처리합니다. PFC는 의사결정, 충동 조절, 실행 기능 등을 담당합니다. 중독 상태에서는 이러한 뇌 영역 간의 연결이 변화하여 보상에 대한 민감도가 증가하고, 충동 조절 능력이 저하됩니다.

#### 예시 (Everyday Example)

- **상황:** 흡연자가 금연을 시도하는 상황을 가정해 봅시다. 흡연은 니코틴이라는 물질을 통해 뇌의 보상 시스템을 활성화시켜 즉각적인 쾌감을 제공합니다. 금연을 시도하면 니코틴 결핍으로 인해 불쾌감, 불안, 초조함 등의 금단 증상이 나타납니다.

- **해석:** 강화학습 관점에서 흡연은 '흡연 -> 쾌감'이라는 긍정적 강화 루프를
```

=== Step B: Explanation ===


```
## Computational Mechanisms × Clinical Features
- Mechanism 1: **높은 할인율 (High Discount Rate, γ)**
  - Clinical Feature: 미래의 부정적인 결과보다 즉각적인 보상을 선호하여 장기적인 건강 문제에도 불구하고 중독 물질을 계속 사용하게 됨.
- Mechanism 2: **시간차 오차 (Temporal Difference Error) 기반의 과도한 가치 학습**
  - Clinical Feature: 중독 물질이나 행동에 대한 예측된 보상과 실제 보상 간의 차이를 과도하게 학습하여, 중독 대상에 대한 갈망과 집착을 강화함.
- Mechanism 3: **전전두피질 기능 저하로 인한 행동 통제 및 의사결정 능력 저하**
  - Clinical Feature: 충동 조절 실패와 잘못된 의사결정으로 인해 중독 행동을 반복하고, 금단 증상에 취약해져 재발 가능성이 높아짐.

```

=== Step C: Explanation ===


```
## Research Directions

- **Q1: 개인별 할인율(γ)의 변동성이 중독 치료 결과에 미치는 영향은 무엇인가?**

  *   **중요성:** 할인율은 개인의 의사결정 과정에 깊숙이 관여하며, 중독 행동의 지속 여부를 결정하는 핵심 요인이다. 개인별 할인율의 차이를 정확히 파악하고, 이를 치료 반응 예측 및 맞춤형 치료 전략 개발에 활용한다면, 중독 치료의 효과를 극대화할 수 있을 것이다. 특히, 인지 행동 치료(CBT)와 같은 치료법의 효과를 개인별 할인율에 맞춰 최적화하는 연구가 필요하다.

- **Q2: 시간차 오차 기반 학습 과정에서 특정 뇌 영역(예: 복측 선조)의 활동 패턴 변화가 중독 심화에 미치는 인과적 영향은 무엇인가?**

  *   **중요성:** 시간차 오차 기반 학습은 중독의 핵심 메커니즘 중 하나이며, 특정 뇌 영역의 활동과 밀접하게 관련되어 있다. 뇌 영상 기술(fMRI, PET)과 뇌 자극 기술(TMS, tDCS)을 결합하여, 특정 뇌 영역의 활동 변화가 중독 행동에 미치는 직접적인 영향을 규명한다면, 중독 치료를 위한 새로운 신경 조절 기술 개발의 가능성을 열 수 있다. 예를 들어, 복측 선조의 과도한 활성화를 억제하는 기술은 중독 환자의 갈망을 효과적으로 감소시킬 수 있을 것이다.

```

## ReACT prompt

In [31]:
# ReAct Prompt
# ====== 환경 설정 ======
import os
import google.generativeai as genai
from IPython.display import Markdown, display
from pathlib import Path

# Colab 환경 체크
try:
    from google.colab import userdata
    IN_COLAB = True
except ImportError:
    IN_COLAB = False

# API 키 설정
if IN_COLAB:
    # Colab 환경에서 API 키 불러오기
    API_KEY = userdata.get("GOOGLE_API_KEY")
else:
    # 로컬 환경에서는 dotenv 사용
    try:
        from dotenv import load_dotenv
        env_path = Path('/Users/yoon/yoonkyung-lee/week3/workshop/.env')
        if env_path.exists():
            load_dotenv(env_path)
        else:
            load_dotenv()
    except ImportError:
        print("python-dotenv가 설치되지 않았습니다. 환경변수에서 직접 읽습니다.")
    API_KEY = os.getenv('GOOGLE_API_KEY')
    if not API_KEY:
        raise ValueError("GOOGLE_API_KEY 환경변수를 설정해주세요")

genai.configure(api_key=API_KEY)
MODEL = "gemini-2.0-flash-exp"  # 필요 시 "gemini-1.5-pro"로 변경 가능

model = genai.GenerativeModel(MODEL)

# Gemini 호출 함수 (간단한 wrapper)
def call_gemini(prompt: str, thinking: int = 0, max_tokens: int = 600, temperature: float = 0.2) -> str:
    """
    prompt: 모델에 전달할 문자열
    thinking: reasoning budget (0=비활성, 1 이상=간단 reasoning 활성)
    return: 모델의 출력 텍스트
    """
    generation_config = genai.GenerationConfig(
        max_output_tokens=max_tokens,
        temperature=temperature,
    )
    
    resp = model.generate_content(
        prompt,
        generation_config=generation_config
    )
    return resp.text

# ====== ReAct 프롬프트 실행 ======
react_prompt = "Use ReAct to help a user find a restaurant: A user is looking for a vegetarian restaurant with outdoor seating in San Francisco. How would you approach this?"

response = call_gemini(react_prompt, thinking=1)  # reasoning 활성화

display(Markdown(response))

E0000 00:00:1758448062.260676 59803079 alts_credentials.cc:93] ALTS creds ignored. Not running on GCP and untrusted ALTS is not enabled.


Okay, here's how I would approach this task using the ReAct (Reasoning and Acting) framework:

**User Input:** "I'm looking for a vegetarian restaurant with outdoor seating in San Francisco."

**ReAct Cycle:**

**1.  Reasoning:**

*   **Goal:** Find a vegetarian restaurant with outdoor seating in San Francisco.
*   **Constraints:**
    *   Must be vegetarian (no meat).
    *   Must have outdoor seating.
    *   Must be located in San Francisco.
*   **Potential Actions:**
    *   Search online for vegetarian restaurants in San Francisco.
    *   Filter the results to show only restaurants with outdoor seating.
    *   Ask the user clarifying questions if needed (e.g., price range, cuisine preference).

**2.  Action:**

*   **Action:** `search[vegetarian restaurants San Francisco outdoor seating]`

**3.  Observation:**

*   **(Assume the search returns the following results - this is a simulated observation):**
    *   "Greens Restaurant (Marina District) - Vegetarian, Outdoor Seating, $$$"
    *   "Gracias Madre (Mission District) - Vegan, Outdoor Seating, $$"
    *   "Souvla (Hayes Valley) - Greek, Outdoor Seating, $ (Note: While Souvla has outdoor seating, it is NOT vegetarian)"
    *   "Cha-Ya (Mission District) - Vegan Japanese, Outdoor Seating, $$"
    *   "Millennium (Closed)"

**4.  Reasoning:**

*   **Analysis of Observation:**
    *   The search returned several potential options.
    *   Greens, Gracias Madre, and Cha-Ya all appear to be vegetarian/vegan and have outdoor seating.
    *   Souvla has outdoor seating but is not vegetarian, so it can be excluded.
    *   Millennium is closed, so it can be excluded.
*   **Next Steps:**
    *   Present the promising options to the user.
    *   Ask if they have any preferences regarding cuisine or price range to narrow down the choices.

**5.  Action:**

*   **Action:** `respond[I found a few vegetarian restaurants in San Francisco with outdoor seating: Greens Restaurant (Marina District), Gracias Madre (Mission District), and Cha-Ya (Mission District).  Greens is vegetarian, while Gracias Madre and Cha-Ya are vegan.  Do you have a preference for cuisine (e.g., Japanese, Mexican, etc.) or price range ($, $$, $$$)?]`

**Next ReAct Cycle (Based on User Response):**

Let's say the user responds: "I'd prefer something in the Mission District and not too expensive."

**1. Reasoning:**

*   **Goal:** Ref

# DIY

In [36]:
react_prompt = """
You are a research assistant. Use a compact ReAct style to reason and act. 
Do NOT reveal long chain-of-thought; give only brief rationale (1–2 lines) per step.

TASK:
- 중독(addiction) 맥락에서 RLDM 메커니즘 2–3개(RPE, model-based vs model-free 가중치, reward sensitivity)를 선정하라.
- 각 메커니즘에 대해 (1) 검증 가능한 가설 1개, (2) behavioral + fMRI 측정지표 2개, (3) 예상 뇌 영역 2개를 제안하라.
- 간단 수식 1개(예: δ = r + γ·V(s′) − V(s)) 포함.
- 혼란변수 2–3개와 완화책 제시.
- 마지막에 5–8줄 요약과 next actions 2개.

FORMAT (print exactly in this order):
Thought: (why these steps? 1–2 lines, Korean with English key terms)
Action: PLAN
Observation: 3–5 bullets of the plan

Thought: (why this design? 1–2 lines)
Action: DESIGN
Observation: 
- Mechanism 1: (가설 / measures / brain areas)
- Mechanism 2: (가설 / measures / brain areas)
- Mechanism 3: (optional)

Thought: (why compute? 1 line)
Action: COMPUTE
Observation: δ = r + γ·V(s′) − V(s)  # brief equation only

Thought: (why these risks? 1 line)
Action: CRITIQUE
Observation: 
- Pitfall → Mitigation
- Pitfall → Mitigation
- Pitfall → Mitigation

Thought: (why this structure? 1 line)
Action: WRITE
Observation:
## 요약(Summary)
- ...
- ...
- ...
- ...
- ...
#### Next actions
- ...
- ...
"""

response = call_gemini(react_prompt, thinking=1)  # reasoning 활성화

display(Markdown(response))

Thought: RLDM 메커니즘 선택 및 실험 설계 계획 수립. 중독 관련 핵심 메커니즘과 검증 가능한 가설, 측정 지표, 뇌 영역을 정의.
Action: PLAN
Observation:
- RLDM 메커니즘 3가지 선정 (RPE, Model-based vs Model-free 가중치, Reward sensitivity)
- 각 메커니즘 별 가설, 행동/fMRI 측정 지표, 예상 뇌 영역 정의
- 혼란 변수 및 완화책 고려
- 요약 및 다음 단계 작성

Thought: 각 메커니즘에 대한 구체적인 가설, 측정 지표, 뇌 영역 설정. 중독 행동과 관련된 신경 메커니즘을 명확히 연결.
Action: DESIGN
Observation:
- Mechanism 1: Reward Prediction Error (RPE)
    - 가설: 중독자는 일반인에 비해 보상 예측 오차에 대한 학습률이 높을 것이다. (Higher learning rate for RPE in addicts)
    - Behavioral: 강화 학습 과제 수행 시 보상 획득 확률 변화에 따른 반응 속도 및 선택 패턴 분석. (Reaction time and choice patterns in reinforcement learning task)
    - fMRI: 강화 학습 과제 수행 시 ventral striatum의 활성화 정도 분석. (Ventral striatum activation during reinforcement learning)
    - Brain Areas: Ventral Striatum, VTA
- Mechanism 2: Model-based vs Model-free 가중치 (Model-based vs Model-free weight)
    - 가설: 중독자는 일반인에 비해 model-free 학습 시스템에 더 의존적일 것이다. (Addicts rely more on model-free learning)
    - Behavioral: 2단계 의사 결정 과제 수행 시 model-based/model-free 전략 사용 비율 분석. (Ratio of model-based/model-free strategies in two-step decision task)
    - fMRI: 2단계 의사 결정 과제 수행 시 prefrontal cortex 및 striatum 활성화 패턴 분석. (Prefrontal cortex and striatum activation patterns during two-step decision task)
    - Brain Areas: Prefrontal Cortex, Dorsolateral Striatum
- Mechanism 3: Reward Sensitivity (보상 민감도)
    - 가설: 중독자는 일반인에 비해 보상 자극에 대한 주관적 가치가 더 높게 평가될 것이다. (Higher subjective value for reward stimuli in addicts)
    - Behavioral: 보상 자극 평가 과제 수행 시 보상에 대한 주관적 가치 평가 점수 비교. (Subjective value rating for reward stimuli)
    - fMRI: 보상 자극 제시 시 orbitofrontal cortex 활성화 정도 분석. (Orbitofrontal cortex activation during reward presentation)
    - Brain Areas: Orb

# Compare through tasks


In [38]:
# ===== Gemini 모델 비교 (모델별 thinking 요구사항 대응) =====
import os, pandas as pd
from IPython.display import display
import google.generativeai as genai
from pathlib import Path

# Colab 환경 체크
try:
    from google.colab import userdata
    IN_COLAB = True
except ImportError:
    IN_COLAB = False

# ---- 환경 ----
if IN_COLAB:
    API_KEY = userdata.get("GOOGLE_API_KEY")
else:
    try:
        from dotenv import load_dotenv
        load_dotenv()  # 현재 디렉토리에서 .env 파일 자동 탐색
    except ImportError:
        print("python-dotenv가 설치되지 않았습니다. 환경변수에서 직접 읽습니다.")
    API_KEY = os.getenv('GOOGLE_API_KEY')
    if not API_KEY:
        raise ValueError("GOOGLE_API_KEY 환경변수를 설정해주세요")

genai.configure(api_key=API_KEY)

# ---- 비교 모델 ----
models = ["gemini-2.0-flash-exp", "gemini-1.5-pro"]

# ---- 태스크 ----
tasks = {
    "Logical Deduction": """If all roses are flowers and some flowers fade quickly, can we conclude that some roses fade quickly? Explain your reasoning.""",
    "Long-Context Understanding": """Summarize the key differences between supervised and unsupervised learning in machine learning.""",
    "Mathematical Proof": """If the area of a circle is 36π square units, what is the radius of the circle?
Show the answer first, then a brief derivation.""",
    "Medical diagnosis": """A 25-year-old woman has frequent, painful urination and a burning sensation while urinating. Which condition is most likely responsible?
Answer succinctly (≤20 words), then list 2 key differentials.""",
    "Legal case": """John signs a written contract with Mary to buy her car for $5,000.
The next day, John changes his mind without any valid reason and refuses to pay.
If Mary sues for breach of contract, who is most likely to win? Explain briefly (≤100 words)."""
}

def analyze_text_with_gemini(model_name: str, prompt: str,
                             temperature: float = 0.2,
                             max_tokens: int = 700) -> str:
    """Gemini API를 사용한 텍스트 분석"""
    try:
        model = genai.GenerativeModel(model_name)
        generation_config = genai.GenerationConfig(
            max_output_tokens=max_tokens,
            temperature=temperature,
        )
        
        response = model.generate_content(
            prompt,
            generation_config=generation_config
        )
        return (response.text or "").strip()
    except Exception as e:
        return f"Error: {e}"

# ---- 실행 ----
results = {}
for task_name, task_prompt in tasks.items():
    print(f"\n📝 Running Task: {task_name}...\n")
    task_results = {}

    for model_name in models:
        print(f"🤖 {model_name} ...")
        response = analyze_text_with_gemini(
            model_name=model_name,
            prompt=task_prompt,
            temperature=0.2 if task_name != "Long-Context Understanding" else 0.3,
            max_tokens=900 if task_name == "Long-Context Understanding" else 700,
        )
        task_results[model_name] = response[:1000]

    results[task_name] = task_results

df = pd.DataFrame.from_dict(results, orient="index")
display(df)


📝 Running Task: Logical Deduction...

🤖 gemini-2.0-flash-exp ...


E0000 00:00:1758450200.034728 59803079 alts_credentials.cc:93] ALTS creds ignored. Not running on GCP and untrusted ALTS is not enabled.


🤖 gemini-1.5-pro ...

📝 Running Task: Long-Context Understanding...

🤖 gemini-2.0-flash-exp ...
🤖 gemini-1.5-pro ...

📝 Running Task: Mathematical Proof...

🤖 gemini-2.0-flash-exp ...
🤖 gemini-1.5-pro ...

📝 Running Task: Medical diagnosis...

🤖 gemini-2.0-flash-exp ...
🤖 gemini-1.5-pro ...

📝 Running Task: Legal case...

🤖 gemini-2.0-flash-exp ...
🤖 gemini-1.5-pro ...


,gemini-2.0-flash-exp,gemini-1.5-pro
Logical Deduction,"Yes, we can conclude that some roses fade quic...","Error: 429 You exceeded your current quota, pl..."
Long-Context Understanding,The key difference between supervised and unsu...,"Error: 429 You exceeded your current quota, pl..."
Mathematical Proof,6 units\n\nThe area of a circle is given by th...,"Error: 429 You exceeded your current quota, pl..."
Medical diagnosis,Most likely a urinary tract infection (UTI).\n...,"Error: 429 You exceeded your current quota, pl..."
Legal case,Mary is most likely to win. John and Mary form...,"Error: 429 You exceeded your current quota, pl..."


# DIY

In [44]:
# 여러 복잡한 어려운 과제를 정의하고 비교 해보기
# 다른 모델의 API KEY가 있을 경우 모델 별로 비교하기
load_dotenv() 
# ---- 비교 모델 ----
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY", "")

def call_openai_model(model_name: str, prompt: str, temperature: float = 0.2, max_tokens: int = 900) -> str:
    try:
        import openai
        openai.api_key = OPENAI_API_KEY
        # Responses API (새 SDK) 사용 예시가 환경에 따라 다를 수 있으므로 가장 단순한 chat.completions 예시:
        client = openai.OpenAI()
        resp = client.chat.completions.create(
            model=model_name,
            messages=[{"role":"user","content":prompt}],
            temperature=temperature,
            max_tokens=max_tokens
        )
        return resp.choices[0].message.content.strip()
    except Exception as e:
        return f"Error(OpenAI): {e}"

models = ["gemini-2.0-flash-exp", "gpt-4o-mini"]

# ---- 태스크 ----
tasks = {
"RLDM_Addiction_Mechanisms": (
        "중독(addiction) 맥락에서 RLDM 메커니즘 2–3개(RPE, model-based vs model-free weight, reward sensitivity)를 요약하고, "
        "각 메커니즘에 대해 (1) 검증 가능한 가설 1개, (2) behavioral + fMRI 측정지표 2개, (3) 예상 뇌 영역 2개를 제시하라. "
        "간단 수식 1개(δ = r + γ·V(s′) − V(s)) 포함. 한국어로, 핵심 용어는 영문 병기."
    ),
    "NF_Pipeline_Design": (
        "금연 맥락의 rtfMRI neurofeedback 파이프라인을 설계하라(측정→추정→피드백→조절→학습). "
        "Target signal 1개(striatal RPE proxy 또는 vmPFC value), 세션 구조(n×duration), 피드백 스케일링/레이턴시, "
        "1차/2차 지표를 제시하고, latency/표적특이성/전략의존성 리스크와 완화책을 간단히 기술. 한국어+영문 병기."
    ),
    "NIBS_Target_Selection": (
        "중독 관련 실행통제/가치평가 결함을 겨냥한 NIBS(TMS/tDCS) 타겟 2곳을 제안하라. "
        "각 타겟에 대해 이론 근거(RLDM 연결), 프로토콜 개요(빈도/강도/세션), 1차 지표(behavior+neural), "
        "안전/윤리 체크포인트를 불릿으로 제시. 한국어+영문 병기."
    )
}

# --- 모델 라우팅 도우미 ---
def run_model(model_name: str, prompt: str, temperature: float = 0.2, max_tokens: int = 900) -> str:
    if model_name.startswith("gemini"):
        # Gemini는 기존 함수 사용
        return analyze_text_with_gemini(
            model_name=model_name,
            prompt=prompt,
            temperature=temperature,
            max_tokens=max_tokens
        )
    else:
        # OpenAI 라우팅 (gpt-*, o*- 계열)
        return call_openai_model(
            model_name=model_name,
            prompt=prompt,
            temperature=temperature,
            max_tokens=max_tokens
        )

# ---- 실행 ----
results = {}
for task_name, task_prompt in tasks.items():
    print(f"\n📝 Running Task: {task_name}...\n")
    task_results = {}

    for model_name in models:
        print(f"🤖 {model_name} ...")
        response = run_model(
            model_name=model_name,
            prompt=task_prompt,
            temperature=0.2,
            max_tokens=700,
        )
        task_results[model_name] = (response or "")[:1000]

    results[task_name] = task_results

df = pd.DataFrame.from_dict(results, orient="index")
pd.set_option("display.max_colwidth", None)  # 열 내용 줄임없이 전체 표시
pd.set_option("display.max_rows", None)      # 행도 다 보여줌
display(df)


📝 Running Task: RLDM_Addiction_Mechanisms...

🤖 gemini-2.0-flash-exp ...
🤖 gpt-4o-mini ...

📝 Running Task: NF_Pipeline_Design...

🤖 gemini-2.0-flash-exp ...
🤖 gpt-4o-mini ...

📝 Running Task: NIBS_Target_Selection...

🤖 gemini-2.0-flash-exp ...
🤖 gpt-4o-mini ...


,gemini-2.0-flash-exp,gpt-4o-mini
RLDM_Addiction_Mechanisms,"Error: 429 You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits.\n* Quota exceeded for metric: generativelanguage.googleapis.com/generate_content_free_tier_requests, limit: 50\nPlease retry in 26.197084731s. [violations {\n quota_metric: ""generativelanguage.googleapis.com/generate_content_free_tier_requests""\n quota_id: ""GenerateRequestsPerDayPerProjectPerModel-FreeTier""\n quota_dimensions {\n key: ""model""\n value: ""gemini-2.0-flash-exp""\n }\n quota_dimensions {\n key: ""location""\n value: ""global""\n }\n quota_value: 50\n}\n, links {\n description: ""Learn more about Gemini API quotas""\n url: ""https://ai.google.dev/gemini-api/docs/rate-limits""\n}\n, retry_delay {\n seconds: 26\n}\n]","Error(OpenAI): Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}"
NF_Pipeline_Design,"Error: 429 You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits.\n* Quota exceeded for metric: generativelanguage.googleapis.com/generate_content_free_tier_requests, limit: 50\nPlease retry in 23.232995614s. [violations {\n quota_metric: ""generativelanguage.googleapis.com/generate_content_free_tier_requests""\n quota_id: ""GenerateRequestsPerDayPerProjectPerModel-FreeTier""\n quota_dimensions {\n key: ""model""\n value: ""gemini-2.0-flash-exp""\n }\n quota_dimensions {\n key: ""location""\n value: ""global""\n }\n quota_value: 50\n}\n, links {\n description: ""Learn more about Gemini API quotas""\n url: ""https://ai.google.dev/gemini-api/docs/rate-limits""\n}\n, retry_delay {\n seconds: 23\n}\n]","Error(OpenAI): Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}"
NIBS_Target_Selection,"Error: 429 You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits.\n* Quota exceeded for metric: generativelanguage.googleapis.com/generate_content_free_tier_requests, limit: 50\nPlease retry in 20.676892226s. [violations {\n quota_metric: ""generativelanguage.googleapis.com/generate_content_free_tier_requests""\n quota_id: ""GenerateRequestsPerDayPerProjectPerModel-FreeTier""\n quota_dimensions {\n key: ""model""\n value: ""gemini-2.0-flash-exp""\n }\n quota_dimensions {\n key: ""location""\n value: ""global""\n }\n quota_value: 50\n}\n, links {\n description: ""Learn more about Gemini API quotas""\n url: ""https://ai.google.dev/gemini-api/docs/rate-limits""\n}\n, retry_delay {\n seconds: 20\n}\n]","Error(OpenAI): Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}"
